In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,log_loss

In [ ]:
df = pd.read_csv("/kaggle/input/mockkk2/train.csv")
df.head()
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df["Status_bin"] = df["Status"].map({"D": 0, "C": 1, "CL": 2})

In [ ]:
X = df.drop(["Status", "Status_bin", "id"], axis=1)

y = df["Status_bin"]

In [ ]:
cat_cols = X.select_dtypes(include=["object"]).columns

label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # ensure string type
    label_encoders[col] = le

In [ ]:
X = X.fillna(X.median(numeric_only=True))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
numeric_cols=df.select_dtypes(include=['int64','float64']).columns
for col in numeric_cols:
    plt.figure(figsize=(6,4))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot of {col}')
    plt.show()

In [ ]:
def cap_outliers(df, cols, lower=1, upper=99):
    for col in cols:
        low_val = df[col].quantile(lower/100)
        high_val = df[col].quantile(upper/100)
        df[col] = df[col].clip(low_val, high_val)
    return df

num_cols = X.select_dtypes(include=['float64','int64']).columns
X = cap_outliers(X, num_cols)

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR

    df[col] = df[col].clip(lower, upper)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
numeric_cols=df.select_dtypes(include=['int64','float64']).columns
for col in numeric_cols:
    plt.figure(figsize=(6,4))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot of {col}')
    plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42, stratify=y)

In [ ]:
# model = RandomForestClassifier(n_estimators=200,random_state=42)
# model.fit(X_train, y_train)



In [ ]:
# from sklearn.calibration import CalibratedClassifierCV

# clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
# model = CalibratedClassifierCV(base_estimator=clf, method='isotonic', cv=5)

# model.fit(X_train, y_train)


In [ ]:
# from sklearn.calibration import CalibratedClassifierCV

# base = lgb.LGBMClassifier(
#     objective='multiclass',
#     metric='multi_logloss',
#     num_class=num_classes,
#     n_estimators=800,
#     learning_rate=0.05,
#     num_leaves=31
# )

# model = CalibratedClassifierCV(base, method='isotonic', cv=5)

# model.fit(X_train, y_train)   # <— this trains AND calibrates


In [ ]:
import lightgbm as lgb
from sklearn.calibration import CalibratedClassifierCV

num_classes = len(np.unique(y_train))

# STEP 1: Define base model (do NOT fit it)
clf = lgb.LGBMClassifier(
    objective='multiclass',
    metric='multi_logloss',
    num_class=num_classes,
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)


model = CalibratedClassifierCV(base_estimator=clf, method='isotonic', cv=5)


model.fit(X_train, y_train)




In [ ]:
y_pred =model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
acc

In [ ]:
y_pred_prob = model.predict_proba(X_test)
log_l = log_loss(y_test,y_pred_prob,labels = model.classes_)
log_l


In [ ]:
df_test = pd.read_csv("/kaggle/input/mockkk2/test.csv")
df_test.head()

In [ ]:
x_test=df_test.drop(["id"], axis=1)

test_ids = df_test["id"]         
x_test = df_test.drop(["id"], axis=1)


for col, le in label_encoders.items():
    if col in x_test.columns:
       
        class_to_int = {cls: idx for idx, cls in enumerate(le.classes_)}
        x_test[col] = x_test[col].astype(str).map(class_to_int)


x_test = x_test.fillna(x_test.median(numeric_only=True))


In [ ]:
y_pred1 = model.predict(x_test)

y_pred1

In [ ]:
y_pred_prob1 = model.predict_proba(x_test)

In [ ]:


class_labels = model.classes_

submission_cols = [f"Status_{label}" for label in class_labels]

submission = pd.DataFrame(y_pred_prob1, columns=submission_cols)

submission.insert(0, "id", test_ids)
submission = submission.rename(columns={
    "Status_0": "Status_D",
    "Status_1": "Status_C",
    "Status_2": "Status_CL"
})

# Reorder columns to: id, Status_C, Status_CL, Status_D
submission = submission[["id", "Status_C", "Status_CL", "Status_D"]]



submission.to_csv("professor.csv", index=False)

print("Submission file saved as submission.csv")
print(submission.head())